In [1]:
%run ComputeRateFromParams.ipynb

Generating cache table of size 1024x1024x128 (~27.0 bits)...


  0%|          | 0/1023 [00:00<?, ?it/s]

Done. Table generation took 6.9 seconds


In [2]:
def compute_func_and_jacobian(f, p: tuple):
    """
    Given some function f and a point p, computes the value of f at p and estimates its Jacobian by looking at p + epsilon e_i.
    """
    DELTA = 1E-10
    base_point = f(*p)
    derivatives = np.zeros(len(p))
    for i in range(len(p)):
        pt = p[:i] + (p[i] + DELTA,) + p[i+1:]
        derivatives[i] = (f(*pt) - base_point) / DELTA
    return (base_point, derivatives)

def get_direction(jacobian: np.ndarray):
    return jacobian / np.linalg.norm(jacobian)

def get_second_derivative(f, f_at_p: float, p: tuple, d: np.ndarray):
    '''
    Given a function f, a starting point p, the value of f at p and a direction d, 
        computes the first and second derivatives of f along d.
    '''
    DELTA = 1E-6
    plus_delta = f(*[p[i] + DELTA*d[i] for i in range(len(p))])
    minus_delta = f(*[p[i] - DELTA*d[i] for i in range(len(p))])
    first_derivative = (plus_delta - minus_delta) / (2 * DELTA)
    second_derivative = (plus_delta + minus_delta - (2 * f_at_p)) / (DELTA ** 2)
    return (first_derivative, second_derivative)
    

def do_optimization_step(f, current_point: tuple, jump_limit: float = 0.1):
    '''
    Given a function f, a starting point and an upper bound on the distance of the jump allowed,
        performs a step of gradient descent to maximize the function f.
    Will ensure that all entries of the output point and all intermediary points are positive.
    Returns as tuple of:
        - The next point
        - The value of f at the starting point
        - The jacobian of f at the starting point
    '''
    bp, jac = compute_func_and_jacobian(f, current_point)
    d = get_direction(jac)
    first_derivative, second_derivative = get_second_derivative(f, bp, current_point, d)
    jump_size = first_derivative / (-(second_derivative))
    jump_size = min(jump_size, jump_limit)
    if jump_size < 0:
        jump_size = jump_limit
    for i in range(len(current_point)):
        if current_point[i] + (jump_size * d[i]) < 0:
            jump_size = min(jump_size, -current_point[i] / (2 * d))
    print(jump_size, jump_size * np.dot(d, jac))
    next_point = tuple([current_point[i] + (jump_size * d[i]) for i in range(len(p))])
    return (next_point, bp, jac)

In [3]:
p = (0.1, 3.5, 0.04)
t0 = time.time()
data = []
for i in it.count():
    p, bp, jac = do_optimization_step(compute_rate_from_beta_g, p, 0.025)
    data.append((p, bp, jac))
    print(p, bp, '%.1f' % (time.time() - t0))

0.025 0.10120844374661511
(0.10908878985364172, 3.5000862963640325, 0.016710808259325655) 0.029886558125224835 22.5
0.007919780746518779 0.01472770806460027
(0.11027986227231976, 3.500092105031849, 0.008881105849541987) 0.11186500339973302 44.5
0.0019014532742437926 0.003937094419998265
(0.11011361680098002, 3.5000917707490586, 0.010775277670060517) 0.11489094236115366 65.1
0.0007411657304146135 0.00035503748602356145
(0.11003958921863505, 3.5000917928654323, 0.011512737198783317) 0.11718386992663432 86.6
6.864684263946156e-05 2.608217069988663e-06
(0.11003218747571385, 3.5000926125232104, 0.01158097891159317) 0.11737098884821616 109.5
5.351321706543294e-07 1.656668049913072e-10
(0.11003205074007548, 3.500092591892921, 0.01158149586826102) 0.11737228939531573 130.2
0.025 8.346665223915924e-06
(0.1341927902297195, 3.506514672325007, 0.011456795277346728) 0.11737228947094157 150.9
0.0018238068803709472 0.0027487896484781537
(0.1340296431739839, 3.50651433880977, 0.01327329038162331) 0.11

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
p =(1.79, 0.1, 0.23, 0.1)
bp, jac = compute_func_and_jacobian(compute_rate_from_beta_g, p)

In [ ]:
d = get_direction(jac)

In [ ]:
get_second_derivative(compute_rate_from_beta_g, bp, p, d)

In [ ]:
np.dot(d, jac)

In [ ]:
np.dot((-0.5 * (_13[0] / _13[1]) * d), jac)

In [ ]:
bp